In [ ]:
import numpy as np
import pandas as pd
import syft as sy

sy.requires("==0.9.1")

In [ ]:
# RFR_URL = "https://reddit-for-researchers.snooguts.net/login"
# guest_client = sy.login_as_guest(url=RFR_URL)
# guest_client.forgot_password(email="prgildersleve@gmail.com")

In [ ]:
# we recommend that you source these values using environment variables
URL = "https://reddit-for-researchers.snooguts.net"
EMAIL = 
PASSWORD = 

# you can provide a "password" keyword argument, but if you don't...
# the browser will prompt you for input
client = sy.login(
    url=URL,
    email=EMAIL,
    password=PASSWORD
)


In [ ]:
client.custom_api.api_endpoints()

In [ ]:
# client.refresh()
client.api.services.reddit.schema()

In [ ]:
sql_query="""
  SELECT subreddit_id, AVG(score) AS average_score
  FROM comments
  GROUP BY subreddit_id
  LIMIT 500
"""

client.api.services.reddit.query.mock(sql_query=sql_query)



In [ ]:
sql_query="""
  SELECT *
  FROM comments
  WHERE body LIKE '%wikipedia%'
  AND created_at >=< '2021-01-01'
  AND created_at < '2022-01-01'
"""

df = client.api.services.reddit.query.mock(sql_query=sql_query)
df

In [ ]:
sql_query="""
  SELECT *
  FROM comments
  WHERE post_id
  IN (
  SELECT id
  FROM posts 
  WHERE (body LIKE '%wikipedia%' OR title LIKE '%wikipedia%')
  )
"""
client.api.services.reddit.query.mock(sql_query=sql_query)



In [ ]:
sql_query="""
  SELECT *
  FROM comments 
  WHERE body LIKE '%facebook%'
  AND created_at >= '2023-01-01'
  AND created_at < '2024-01-01'
"""

client.api.services.reddit.query.mock(sql_query=sql_query)


In [ ]:
sql_query="""
  SELECT *
  FROM comments 
  WHERE body LIKE '%wikipedia%'
  AND created_at >= '2020-01-01'
  AND created_at < '2021-01-01'
"""

client.api.services.reddit.submit_query(
    func_name="pg_wiki_comments_2020",
    sql_query=sql_query
)

In [ ]:
queries_dict = {}
for year in range(2020, 2024):
    for month in range(1, 13):
        start_date = f"{year}-{month:02d}-01"
        if month == 12:
            end_date = f"{year + 1}-01-01"
        else:
            end_date = f"{year}-{month + 1:02d}-01"
        
        sql_query = f"""
        SELECT *
        FROM comments
        WHERE body LIKE '%wikipedia%'
        AND created_at >= '{start_date}'
        AND created_at < '{end_date}'
        """
        key = f"pg_wiki_comments_{year}{month:02d}"
        queries_dict[key] = sql_query

queries_dict

In [ ]:
# submit all queries in the dictionary
for key, sql_query in queries_dict.items():
    print(key)
    if '2020' in key:
        continue
    client.api.services.reddit.submit_query(
        func_name=key,
        sql_query=sql_query
    )

In [ ]:
key

In [ ]:
client.api.services.reddit.query.mock(sql_query=sql_query)

In [ ]:
sql_query="""
  SELECT *
  FROM comments
  WHERE post_id
  IN (
  SELECT id
  FROM posts 
  WHERE (body LIKE '%wikipedia%' OR title LIKE '%wikipedia%')
  )
"""

client.api.services.reddit.submit_query(
    func_name="pg_wiki_replies",
    sql_query=sql_query
)

In [ ]:
sql_query="""
  SELECT *
  FROM posts 
  WHERE (body LIKE '%wikipedia%' OR title LIKE '%wikipedia%')
  AND created_at >= '2021-01-01'
  AND created_at < '2022-01-01'
"""

client.api.services.reddit.submit_query(
    func_name="pg_wiki_2021",
    sql_query=sql_query
)

In [ ]:
client.refresh()
client.requests


In [ ]:
import time

y_id = {2020: '4515465e7f39480cb163f8c880d0b9dd', 2021: '8828789e73774282a4b9842f86032615',
 2022: 'ad435289e53d4393ba6556cf69a5a032', 2023: '27922f61db4644338f0c2e917d6a7a38'}

wikidf = []
for y, id in y_id.items():
    while True:
        print(y)
        try:
            request = client.api.services.request.get_by_uid(uid=sy.UID(id))
            job = request.code(blocking=False)
            results = job.wait()
            df = results.get()
            wikidf.append(df)
            break
        except AttributeError as ex:
            print(ex)
            time.sleep(10)
            client = sy.login(
                url=URL,
                email=EMAIL,
                password=PASSWORD
            )

wikidf = pd.concat(wikidf, ignore_index=True)


In [ ]:
import time

while True:   
    try:
        request = client.api.services.request.get_by_uid(uid=sy.UID('a4a5e43a1aea4c9e87f857d40e59e1bb'))
        job = request.code(blocking=False)
        results = job.wait()
        df = results.get()
        break
    except AttributeError as ex:
        print(ex)
        time.sleep(10)
        client = sy.login(
            url=URL,
            email=EMAIL,
            password=PASSWORD
        )

In [ ]:
results.dict()

In [ ]:
client.requests

In [ ]:
request = client.api.services.request.get_by_uid(uid=sy.UID('d02becf533fc4085911783f6c48d1f9e'))
job = request.code(blocking=False)
results = job.wait()
df = results.get()

In [ ]:
df

In [ ]:
request = client.api.services.request.get_by_uid(uid=sy.UID('2bf9dd73b0d34659a08dec5185f5694c'))
job = request.code(blocking=False)
results = job.wait()
df = results.get()

In [ ]:
results.dict()

In [ ]:
wikidf.isnull().sum()

In [ ]:
ex

In [ ]:
for c in ['score', 'gildings', 'num_comments']:
    wikidf[c] = wikidf[c].astype(np.int64)
for c in ['nsfw', 'self', 'video', 'locked', 'spoiler', 'sticky']:
    wikidf[c] = wikidf[c].astype(np.bool_)
for c in ['created_at', 'updated_at']:
    wikidf[c] = pd.to_datetime(wikidf[c])


In [ ]:
wikidf.to_hdf('wikidf.h5', key='df')

In [ ]:
len(wikidf)

In [ ]:
wikidf['created_at'].hist(bins=1000)

In [ ]:
import re

# Regex pattern to identify Wikipedia links
pattern = r'([a-z]{0,1,2,3}\.)?wikipedia\.org/wiki/[^\s]+'

# Example usage
text = "Check out this Wikipedia link: https://en.wikipedia.org/wiki/Artificial_intelligence and this one: https://fr.wikipedia.org/wiki/Intelligence_artificielle"
matches = re.findall(pattern, text)
print(matches)

In [ ]:
# pattern = r'wikipedia\.org/wiki/[^\s]+'
wikidf['body'].str.findall(pattern).explode().value_counts()

In [ ]:
print(wikidf.loc[4, 'body'])

In [ ]:
text = "Check out this Wikipedia link: [fjgsjf](https://en.wikipedia.org/wiki/Artificial_intelligence) and this one: https://fr.wikipedia.org/wiki/Intelligence_artificielle"
pattern = r'\[([^\]]+)\]\(https?://[a-z]{2,3}\.wikipedia\.org/wiki/[^\s)]+\)'
matches = re.findall(pattern, text)
matches

In [ ]:
pattern = r'\S*wikipedia.org\S*'
uls = wikidf['body'].str.findall(pattern).explode().value_counts()

In [ ]:
print('\n'.join(uls.index[:100]))

In [ ]:
import urlextract

t = '[Houseparty](https://en.wikipedia.org/wiki/Houseparty_(app)),'

url_extractor = urlextract.URLExtract()
urls = url_extractor.find_urls(t)
urls

In [ ]:
urls =  """GROUPS](https://en.wikipedia.org/wiki/Warez_scene_)**
24h](https://en.wikipedia.org/wiki/Special:RecentChangesLinked?hidebots=1&hideminor=1&hidenewpages=1&hidecategorization=1&hideWikibase=1&hidelog=1&target=User%3AGuarapiranga%2Fsandbox%2F1&limit=1000&days=7&enhanced=1&urlversion=2)
page)](https://wikipedia.org/wiki/Special:Random)
[https://en.wikipedia.org/wiki/Race\_and\_ethnicity\_in\_the\_United\_States](https://en.wikipedia.org/wiki/Race_and_ethnicity_in_the_United_States)
contemporary](https://en.wikipedia.org/wiki/Soft_adult_contemporary)[Bell
Radio](https://en.wikipedia.org/wiki/Bell_Media_Radio)[French](https://en.wikipedia.org/wiki/French_language)
Media](https://en.wikipedia.org/wiki/Rogers_Media)FM
Falls](https://en.wikipedia.org/wiki/Smiths_Falls,_Ontario)FM
Corporation](https://en.wikipedia.org/wiki/Canadian_Broadcasting_Corporation)FM
Radio](https://en.wikipedia.org/wiki/Bell_Media_Radio)FM
Corporation](https://en.wikipedia.org/wiki/Canadian_Broadcasting_Corporation)[French](https://en.wikipedia.org/wiki/French_language)FM
Media](https://en.wikipedia.org/wiki/Rogers_Media)licensed
Group](https://en.wikipedia.org/wiki/Stingray_Group)FM
Media](https://en.wikipedia.org/wiki/RNC_Media)[French](https://en.wikipedia.org/wiki/French_language)
Entertainment](https://en.wikipedia.org/wiki/Corus_Entertainment)FM
[https://en.wikipedia.org/wiki/List\_of\_suicide\_crisis\_lines](https://en.wikipedia.org/wiki/List_of_suicide_crisis_lines)
[Houseparty](https://en.wikipedia.org/wiki/Houseparty_(app)),
needed*](https://en.wikipedia.org/wiki/Wikipedia:Citation_needed)\]
App](https://en.wikipedia.org/wiki/Cash_App)
https://en.wikipedia.org/wiki/Truth#Mathematics
https://en.m.wikipedia.org/wiki/Demographics_of_Ontario
speech](https://en.wikipedia.org/wiki/Hate_speech)
Teahouse](https://en.wikipedia.org/wiki/Wikipedia:Teahouse)
Contents](https://en.wikipedia.org/wiki/Help:Contents)
[Wikipedia.org](http://www.wikipedia.org)
[ISBN](https://en.wikipedia.org/wiki/ISBN_(identifier))
Wikipedia](http://en.wikipedia.org/wiki/List_of_Major_League_Soccer_seasons)
Country](https://en.wikipedia.org/wiki/Pure_Country_(radio_network))
OBJKECT[B](https://en.wikipedia.org/wiki/The_Shadow_of_the_Torturer)UILDING
music](https://en.wikipedia.org/wiki/Country_music)[Bell
106.9[CKQB-FM](https://en.wikipedia.org/wiki/CKQB-FM)Jump
radio](https://en.wikipedia.org/wiki/Contemporary_hit_radio)[Corus
93.9[CKKL-FM](https://en.wikipedia.org/wiki/CKKL-FM)[Pure
100.3[CJMJ-FM](https://en.wikipedia.org/wiki/CJMJ-FM)Majic
radio](https://en.wikipedia.org/wiki/Community_radio)Radio
contemporary](https://en.wikipedia.org/wiki/Hot_adult_contemporary)[Rogers
Ottawa](https://en.wikipedia.org/wiki/University_of_Ottawa)FM
FM](https://en.wikipedia.org/wiki/Rouge_FM)[soft
radio](https://en.wikipedia.org/wiki/Campus_radio)[University
89.1[CHUO-FM](https://en.wikipedia.org/wiki/CHUO-FM)CHUO
rock](https://en.wikipedia.org/wiki/Modern_rock)[Stingray
hits](https://en.wikipedia.org/wiki/Adult_hits)[Rogers
88.5[CILV-FM](https://en.wikipedia.org/wiki/CILV-FM)Live
radio](https://en.wikipedia.org/wiki/Contemporary_hit_radio)[Stingray
105.3[CISS-FM](https://en.wikipedia.org/wiki/CISS-FM)Kiss
radio](https://en.wikipedia.org/wiki/Talk_radio)[Cogeco](https://en.wikipedia.org/wiki/Cogeco)[French](https://en.wikipedia.org/wiki/French_language)
104.7[CKOF-FM](https://en.wikipedia.org/wiki/CKOF-FM)104,7[Talk
radio](https://en.wikipedia.org/wiki/Contemporary_hit_radio)[Bell
104.1[CKTF-FM](https://en.wikipedia.org/wiki/CKTF-FM)[Énergie](https://en.wikipedia.org/wiki/%C3%89nergie)[contemporary
Music](https://en.wikipedia.org/wiki/CBC_Music)[public](https://en.wikipedia.org/wiki/Public_broadcasting)
FM](https://en.wikipedia.org/wiki/Rythme_FM)[soft
89.9[CIHT-FM](https://en.wikipedia.org/wiki/CIHT-FM)Hot
Première](https://en.wikipedia.org/wiki/Ici_Radio-Canada_Premi%C3%A8re)[news](https://en.wikipedia.org/wiki/News)/[talk](https://en.wikipedia.org/wiki/Talk_radio)[Canadian
94.9[CIMF-FM](https://en.wikipedia.org/wiki/CIMF-FM)[Rouge
90.7[CBOF-FM](https://en.wikipedia.org/wiki/CBOF-FM)[Ici
92.3[CJET-FM](https://en.wikipedia.org/wiki/CJET-FM)[Jack
92.7[CJVN-FM](https://en.wikipedia.org/wiki/CJVN-FM)[Christian
radio](https://en.wikipedia.org/wiki/Christian_radio)Fiston
Kalambay[French](https://en.wikipedia.org/wiki/French_language)
Musique](https://en.wikipedia.org/wiki/Ici_Musique)[public](https://en.wikipedia.org/wiki/Public_broadcasting)
93.1[CKCU-FM](https://en.wikipedia.org/wiki/CKCU-FM)CKCU
radio](https://en.wikipedia.org/wiki/Campus_radio)[Carleton
One](https://en.wikipedia.org/wiki/CBC_Radio_One)[news](https://en.wikipedia.org/wiki/News)/[talk](https://en.wikipedia.org/wiki/Talk_radio)[Canadian
91.5[CBO-FM](https://en.wikipedia.org/wiki/CBO-FM)[CBC
94.5[CJFO-FM](https://en.wikipedia.org/wiki/CJFO-FM)[community
d'Ottawa[French](https://en.wikipedia.org/wiki/French_language)
107.9[CKDJ-FM](https://en.wikipedia.org/wiki/CKDJ-FM)CKDJ
rock](https://en.wikipedia.org/wiki/Mainstream_rock)[Rogers
106.1[CHEZ-FM](https://en.wikipedia.org/wiki/CHEZ-FM)Chez
FM](https://en.wikipedia.org/wiki/Jack_FM)[adult
103.3[CBOQ-FM](https://en.wikipedia.org/wiki/CBOQ-FM)[CBC
97.1[CHLX-FM](https://en.wikipedia.org/wiki/CHLX-FM)[Rythme
University](https://en.wikipedia.org/wiki/Carleton_University)FM
96.5[CFTX-FM](https://en.wikipedia.org/wiki/CFTX-FM)POP
Communications](https://en.wikipedia.org/wiki/Evanov_Communications)FM
98.5[CJWL-FM](https://en.wikipedia.org/wiki/CJWL-FM)The
International](https://en.wikipedia.org/wiki/CHIN_Radio/TV_International)FM
[community](https://en.wikipedia.org/wiki/Community_radio)[CHIN
99.1[CHRI-FM](https://en.wikipedia.org/wiki/CHRI-FM)CHRI
97.9[CJLL-FM](https://en.wikipedia.org/wiki/CJLL-FM)CHIN
95.7[CFPO-FM](https://en.wikipedia.org/wiki/CFPO-FM)Elmnt
contemporary](https://en.wikipedia.org/wiki/Adult_contemporary)[Aboriginal
Network](https://en.wikipedia.org/wiki/Aboriginal_Peoples_Television_Network)FM
[Kemptville](https://en.wikipedia.org/wiki/Kemptville,_Ontario)FM
music](https://en.wikipedia.org/wiki/Christian_music)Christian
99.7[CJOT-FM](https://en.wikipedia.org/wiki/CJOT-FM)[boom
99.7](https://en.wikipedia.org/wiki/Boom_FM)[Classic
standards](https://en.wikipedia.org/wiki/Pop_standards)[Evanov
Group](https://en.wikipedia.org/wiki/Vista_Broadcast_Group)licensed
music](https://en.wikipedia.org/wiki/Country_music)[Rogers
hits](https://en.wikipedia.org/wiki/Classic_hits)[RNC
contemporary](https://en.wikipedia.org/wiki/Soft_adult_contemporary)[RNC
102.5[CBOX-FM](https://en.wikipedia.org/wiki/CBOX-FM)[Ici
Rock](https://en.wikipedia.org/wiki/Mainstream_Rock)Torres
101.7[CIDG-FM](https://en.wikipedia.org/wiki/CIDG-FM)Rebel
contemporary](https://en.wikipedia.org/wiki/Adult_contemporary)[Vista
97.5[CKKV-FM](https://en.wikipedia.org/wiki/CKKV-FM)The
101.1[CKBY-FM](https://en.wikipedia.org/wiki/CKBY-FM)Country
Hits](https://en.wikipedia.org/wiki/Classic_Hits)[Corus
College](https://en.wikipedia.org/wiki/Algonquin_(College))
en.wikipedia.org/wiki/CBOQ-FM
wikipedia.org/wiki/CBOQ-(FM)
    """


# Define the regex pattern to extract Wikipedia URLs
pattern = r'(https?://)?((?:\w+\.)?wikipedia\.org/?\S*)'

# Example regex pattern to match URLs with or without https, and handle parentheses
pattern = r'(\(?https?://)?((?:\w+\.)?wikipedia\.org/?[\S]*)\)?'

# Find all matches in the urls string
wikipedia_urls = re.findall(pattern, urls)
wikipedia_urls

In [ ]:
import markdown
from bs4 import BeautifulSoup

In [ ]:
import markdown
from bs4 import BeautifulSoup

# Sample text containing Markdown-style links
text = """
Check out [OpenAI](https://www.openai.com) and [Python](https://www.python.org) for more information.
Also, [Link with [nested] brackets](https://example.com/page(yes)) and [Link with escaped \\] bracket](https://example.com).
"""

# Convert Markdown to HTML
soup = BeautifulSoup(markdown.markdown(text), 'html.parser')

# Find all links
for a in soup.find_all('a'):
    displayed_text = a.text
    url = a['href']
    print(f"Displayed text: {displayed_text}, URL: {url}")


In [ ]:
def extract_links(text):
    # Convert Markdown to HTML
    soup = BeautifulSoup(markdown.markdown(text), 'html.parser')
    # Find all links
    links = [a['href'] for a in soup.find_all('a')]
    return links

In [ ]:
import urllib.parse

def extract_links(text):
    # Convert Markdown to HTML
    soup = BeautifulSoup(markdown.markdown(text), 'html.parser')
    # Find all links
    links = [a.get("href") for a in soup.find_all('a')]
    # Normalize URLs
    llinks = []
    for link in links:
        try:
            if link:
                ll = urllib.parse.urlparse(link).geturl()
                if not urllib.parse.urlparse(ll).scheme:
                    ll = 'https://' + ll
                if urllib.parse.urlparse(ll).netloc[-13:] == 'wikipedia.org':
                    llinks.append(ll)
        except Exception as ex:
            print(link, ex)
    
    # links = [urllib.parse.urlparse(link) for link in links if link]
    # links = [l.geturl() for l in llinks if l.netloc[-13:] == 'wikipedia.org']
    return llinks

def extract_links2(text):
    # Convert Markdown to HTML
    soup = BeautifulSoup(markdown.markdown(text), 'html.parser')
    # Find all links
    for a in soup.find_all('a'):
        a.replace_with('')

    pattern = r'\S*.wikipedia\.org\S*'
    matches = re.findall(pattern, soup.text)
    
    return matches

In [ ]:
for a in soup.find_all('a'):
    a.replace_with('')

pattern = r'\S*.wikipedia\.org\S*'
matches = re.findall(pattern, soup.text)
matches

In [ ]:
wikidf['body'].iloc[:100000].fillna('').apply(extract_links2).explode().value_counts()

In [ ]:
l = '[https://en.wikipedia.org/wiki/WarGames_match](https://en.wikipedia.org/wiki/WarGames_match)'
ll = urllib.parse.urlparse(l).geturl()
# if not urllib.parse.urlparse(ll).scheme:
#     ll = 'https://' + ll
ll

In [ ]:
urllib.parse.urlparse('en.wikipedia.org').geturl()

In [ ]:
wikidf['wikilinks'] = wikidf['body'].fillna('').apply(extract_links)

In [ ]:
wikidf['wikilinks'].str.len()

In [ ]:
soup = BeautifulSoup(markdown.markdown(t), 'html.parser')
# Find all links
links = [a.get("href") for a in soup.find_all('a')]
llinks = = []
for link in links:
    try:
        if link:
            llinks.append(urllib.parse.urlparse(link).geturl())
    except:
        print(link)
links = [urllib.parse.urlparse(link) for link in links[71:72] if link]
# links = [l.geturl() for l in links if l.netloc[-13:] == 'wikipedia.org']

In [ ]:
soup.find_all('a')[71]

In [ ]:
text = wikidf['body'].iloc[1804]
soup = BeautifulSoup(markdown.markdown(text), 'html.parser')
# Find all links
links = [a.get("href") for a in soup.find_all('a')]


In [ ]:
print(text[1000:2000])

In [ ]:
urllib.parse.urlparse('https://en.wikipedia.org/wiki/Tupolev_Tu-85')